In [110]:
import pandas as pd

In [112]:
X_full = pd.read_csv('msk_test.csv')
X_full.shape

(1472, 29)

In [ ]:
X = X_full.drop(columns={'price', 'address'}, axis=1)

In [ ]:
y = X_full['price']

In [ ]:
X.shape

(5978, 24)

In [ ]:
X.describe()

,total_area,kitchen_area,living_area,rooms_count,floor,floors_number,build_date,isСomplete,complitation_year,balcony,longitude,latitude,passenger_elevator,cargo_elevator,is_apartments,is_auction
count,5978.000000,3702.000000,3834.000000,5248.000000,5978.000000,5978.000000,1497.000000,3797.000000,3869.000000,1405.00000,5978.000000,5978.000000,3254.000000,2693.000000,5040.000000,5978.000000
mean,66.546173,14.366667,33.818258,2.102515,7.840248,16.875878,1999.291917,0.134317,2024.574309,0.80000,50.760080,56.604465,2.464352,2.580765,0.063492,0.238207
std,59.392769,10.142335,33.068501,1.043265,7.354915,9.565322,31.715727,0.341037,1.351384,0.69839,17.221261,1.593255,3.030908,3.267750,0.243870,0.426022
min,12.000000,1.000000,2.100000,1.000000,-2.000000,1.000000,1770.000000,0.000000,2013.000000,0.00000,29.514256,54.856743,0.000000,0.000000,0.000000,0.000000
25%,38.615000,8.000000,16.000000,1.000000,3.000000,9.000000,1985.000000,0.000000,2024.000000,0.00000,37.565079,55.731537,1.000000,1.000000,0.000000,0.000000
50%,53.185000,12.700000,26.600000,2.000000,5.000000,17.000000,2013.000000,0.000000,2025.000000,1.00000,44.075142,56.219562,2.000000,1.000000,0.000000,0.000000
75%,73.852500,17.900000,39.900000,3.000000,11.000000,24.000000,2020.000000,0.000000,2025.000000,1.00000,60.603778,56.819113,3.000000,3.000000,0.000000,0.000000
max,1285.800000,168.700000,700.000000,6.000000,82.000000,95.000000,2026.000000,1.000000,2028.000000,4.00000,83.123521,60.194665,30.000000,30.000000,1.000000,1.000000


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
categorial_cols = [cname for cname in X_train.columns if
                    X_train[cname].nunique() < 10 and 
                    X_train[cname].dtype == "object"]

In [ ]:
numerical_cols = [cname for cname in X_train.columns if 
                X_train[cname].dtype in ['int64', 'float64']]

In [ ]:
my_cols = categorial_cols + numerical_cols
X_train = X_train[my_cols].copy()
X_valid = X_val[my_cols].copy()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


In [ ]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy="median")),
])

In [ ]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorial_cols)
    ])

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_jobs=-1, random_state=42, verbose=1)

In [ ]:
clf_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'model__n_estimators' : list(range(100, 501, 50)),
          'model__max_depth' :list(range(1, 8, 2)),
          }
best_clf = GridSearchCV(clf_pipeline, params, cv=5, verbose=1)

In [ ]:
best_clf.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parall

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median'))]),
                                                                         ['total_area',
                                                                          'kitchen_area',
                                                                          'living_area',
                                                                          'rooms_count',
                                                                          'floor',
                                                                          'floors_number',
                                                                          'build_date',
                                                                          'isСomplete',
                                                                          'complitation_year',
                                                                          'balcony',
                                                                          'longitude',
                                                                          'latitude',
                                                                          'passenger_elevator',
                                                                          'cargo_...
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['region',
                                                                          'house_material',
                                                                          'parking',
                                                                          'decoration'])])),
                                       ('model',
                                        RandomForestRegressor(n_jobs=-1,
                                                              random_state=42,
                                                              verbose=1))]),
             param_grid={'model__max_depth': [1, 3, 5, 7],
                         'model__n_estimators': [100, 150, 200, 250, 300, 350,
                                                 400, 450, 500]},
             verbose=1)

In [ ]:
predicitions = best_clf.predict(X_valid)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished


In [ ]:
best_clf

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median'))]),
                                                                         ['total_area',
                                                                          'kitchen_area',
                                                                          'living_area',
                                                                          'rooms_count',
                                                                          'floor',
                                                                          'floors_number',
                                                                          'build_date',
                                                                          'isСomplete',
                                                                          'complitation_year',
                                                                          'balcony',
                                                                          'longitude',
                                                                          'latitude',
                                                                          'passenger_elevator',
                                                                          'cargo_...
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['region',
                                                                          'house_material',
                                                                          'parking',
                                                                          'decoration'])])),
                                       ('model',
                                        RandomForestRegressor(n_jobs=-1,
                                                              random_state=42,
                                                              verbose=1))]),
             param_grid={'model__max_depth': [1, 3, 5, 7],
                         'model__n_estimators': [100, 150, 200, 250, 300, 350,
                                                 400, 450, 500]},
             verbose=1)

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
print('MAE:', mean_absolute_error(y_val, predicitions))
print('R^2:', r2_score(y_val, predicitions))
print('MAPE:', mean_absolute_percentage_error(y_val, predicitions))
print('RMSE:', (mean_squared_error(y_val, predicitions)) ** (1/2))


MAE: 10420449.648660738
R^2: 0.6475610703818333
MAPE: 0.30855819284640185
RMSE: 56607452.964881115
